In [244]:
import pandas as pd
import os
import cv2 as cv
import numpy as np
from shutil import copyfile
from IPython.display import clear_output

In [88]:
img = cv.imread("positives/20191225T044013+0900_infrared_image.png")
cv.imshow("Original", img)
cv.waitKey(0)
cv.destroyWindow("Original")
cv.waitKey(1)

-1

## **Contour detection**

In [137]:
print(images[:3])

['20191228T082803+0900_infrared_image.png', '20200105T010758+0900_infrared_image.png', '20200103T094445+0900_infrared_image.png']


In [186]:
img1 = cv.imread("positives/"+images[0], cv.IMREAD_GRAYSCALE)

In [187]:
adaptive = cv.adaptiveThreshold(src=img1, maxValue=255, adaptiveMethod=cv.ADAPTIVE_THRESH_GAUSSIAN_C, thresholdType=cv.THRESH_BINARY_INV, blockSize=1001, C=5)
cv.imshow("Adaptive", adaptive)
cv.waitKey(0)
cv.destroyWindow("Adaptive")
cv.waitKey(1)

-1

In [103]:
# Apply binary thresholding
img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
ret, thresh = cv.threshold(img_gray, 90, 255, cv.THRESH_BINARY)
thresh = cv.bitwise_not(thresh)
thresh = thresh.astype(np.uint8)

cv.imshow("Binary thresh", thresh)
cv.waitKey(0)
cv.destroyWindow("Binary thresh")
cv.waitKey(1)

-1

In [104]:
contours, hierarchy = cv.findContours(image=thresh, mode=cv.RETR_TREE, method=cv.CHAIN_APPROX_NONE)

# Filter contours that are too small or too large
filtered_contours = []
for con in contours:
    if cv.contourArea(con) > 8000 and cv.contourArea(con) < 13000:
        filtered_contours.append(con)

In [105]:
contour_img = img.copy()
cv.drawContours(contour_img, filtered_contours, contourIdx=-1, color=(0, 255, 0), thickness=1, lineType=cv.LINE_AA)
cv.imshow("None approximation", contour_img)
cv.waitKey(0)
cv.destroyWindow("None approximation")
cv.waitKey(1)

-1

#### Run through many images and display contours

In [269]:
images = os.listdir("positives/")
total_imgs = 0
true_positive = 0                   # 1 Weigher found
false_positive = 0                  # 2 False object found
false_negative = 0                  # 3 Weigher not found
false_and_true_positive = 0         # 4 Weigher and false object found
false_negative_and_positive = 0     # 5 False object found and weigher not found
true_negative = 0                   # 6 Weigher not in image
true_negative_and_false_positive = 0# 7 Weigher is not there but false object is found

for im in images:
    if im.startswith(".") or im.startswith("resize"):
        continue
    else:
        # Img preprocessing
        oimg = cv.imread("positives/"+im)
        limg = cv.cvtColor(oimg, cv.COLOR_BGR2GRAY)
        # ret, thresh = cv.threshold(limg, 90, 255, cv.THRESH_BINARY)
        # thres = cv.bitwise_not(thresh)
        thresh = cv.adaptiveThreshold(src=limg, maxValue=255, adaptiveMethod=cv.ADAPTIVE_THRESH_GAUSSIAN_C, thresholdType=cv.THRESH_BINARY_INV, blockSize=1001, C=5)
        thresh = thresh.astype(np.uint8)

        # Find countours
        contours, hierarchy = cv.findContours(image=thresh, mode=cv.RETR_TREE, method=cv.CHAIN_APPROX_NONE)

        # Filter contours by size
        filtered_contours = []
        for con in contours:
            hull = cv.convexHull(con)
            hull_area = cv.contourArea(hull)
            con_area = cv.contourArea(con)
            x, y, w, h = cv.boundingRect(con)

            # Continue if hull area is too small or too large
            if hull_area > 40000 or hull_area < 11000:
                continue
            # Continue if hull is too close to image corders
            if (x == 0 or x > 960) or (y == 0 or y > 500):
                continue
            
            solidity = con_area / hull_area
            aspect_ratio = w / h
            if aspect_ratio > 0.8 and aspect_ratio < 1.25 and solidity > 0.5:
                filtered_contours.append(con)
                print("\tx, y, w, h:\t", x, y, w, h)
                print("\tAspect ratio:\t", aspect_ratio)
                print("\tSolidity\t", solidity)
                print("\tHull area\t", hull_area, "\n")
        print(len(filtered_contours), "out of", len(contours), "contours passed filtering\n")

        # contour_img = limg.copy()
        # Display discarded contours in red
        cv.drawContours(oimg, contours, contourIdx=-1, color=(0, 0, 255), thickness=1, lineType=cv.LINE_AA)
        # Display image with filtered contours in green
        cv.drawContours(oimg, filtered_contours, contourIdx=-1, color=(0, 255, 0), thickness=1, lineType=cv.LINE_AA)
        # Display bounding box in blue
        for con in filtered_contours:
            x, y, w, h = cv.boundingRect(con)
            cv.rectangle(oimg, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv.putText(oimg, "WEIGHER", (x, y-8), cv.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2, cv.LINE_AA)

        cv.imshow("Contours", oimg)
        k = cv.waitKey(0)
        cv.destroyWindow("Contours")
        cv.waitKey(1)
        clear_output(wait=True)

        # Quit if "q" is pressed
        if k == ord("q"):
            break
        elif k == ord("1"):
            true_positive += 1
        elif k == ord("2"):
            false_positive += 1
        elif k == ord("3"):
            false_negative += 1
        elif k == ord("4"):
            false_and_true_positive += 1
        elif k == ord("5"):
            false_negative_and_positive += 1
        elif k == ord("6"):
            true_negative += 1
        elif k == ord("7"):    
            true_negative_and_false_positive += 1
        total_imgs += 1

print("Total images:\t\t", total_imgs)
print("True positives:\t\t", true_positive + false_and_true_positive)
print("False positives:\t", false_positive + false_negative_and_positive + false_and_true_positive + true_negative_and_false_positive)
print("True negatives:\t\t", true_negative + true_negative_and_false_positive)
print("False negatives:\t", false_negative + false_negative_and_positive, "\n")
print("Detection rate:\t\t", true_positive / total_imgs)
print("False positive rate:\t", (false_positive + false_and_true_positive) / total_imgs)
print("False negative rate:\t", (false_negative + false_negative_and_positive) / total_imgs)


Total images:		 1
True positives:		 1
False positives:	 0
True negatives:		 0
False negatives:	 0 

Detection rate:		 1.0
False positive rate:	 0.0
False negative rate:	 0.0


**Parameters:** <br>
Hull area:&emsp;&emsp;&ensp;12.000 < hull_area < 40.000 <br>
Solidity:&emsp;&emsp;&emsp;solidity > 0.6 <br>
Aspect ratio:&emsp;0.8 < aspect_ratio < 1.2 <br>

**Results** <br>
Total images:&emsp;&emsp;200 <br>
True positives:&emsp;&ensp;70 <br>
False positives:&emsp;&ensp;0 <br>
True negatives:&emsp;&ensp;0 <br>
False negatives:&emsp;130 <br>
<br>

**Parameters:** <br>
Hull area:&emsp;&emsp;&ensp;11.000 < hull_area < 40.000 <br>
Solidity:&emsp;&emsp;&emsp;solidity > 0.5 <br>
Aspect ratio:&emsp;0.6 < aspect_ratio < 1.7 <br>

**Results** <br>
Total images:&emsp;&emsp;200 <br>
True positives:&emsp;&ensp;97 <br>
False positives:&emsp;&ensp;8 <br>
True negatives:&emsp;&ensp;0 <br>
False negatives:&emsp;95<br>
The second run resulted in significantly larger bounding boxes being accepted

## **Hough Transformation**

In [98]:
cimg = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])
cimg = cv.filter2D(src=cimg, ddepth=-1, kernel=kernel)


circles = cv.HoughCircles(cimg, method=cv.HOUGH_GRADIENT_ALT, dp=1, minDist=10, param1=20, param2=0.90, minRadius=30, maxRadius=100)
len(circles[0])

1

In [99]:
circles = np.uint16(np.around(circles))
for i in circles[0, :]:
    # draw outer circle
    cv.circle(cimg, center=(i[0], i[1]), radius=i[2], color=(0, 0, 255), thickness=2)
    # draw center
    cv.circle(cimg, center=(i[0], i[1]), radius=3, color=(0, 255, 0), thickness=3)

cv.imshow("Detected circles", cimg)
cv.waitKey(0)
cv.destroyWindow("Detected circles")
cv.waitKey(1)

-1

## **Blob detection**

In [40]:
bimg = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
params = cv.SimpleBlobDetector_Params()
# Circularity parameters
params.filterByCircularity = False
# params.minCircularity = 0.8
# Threshold parameters
# params.minThreshold = 70
# params.maxThreshold = 130
# params.thresholdStep = 1
# Area parameters
params.filterByArea = True
params.minArea = 1000
params.maxArea = 300000
# Color parameters
params.filterByColor = False
# params.blobColor = 255
# Inertia parameters
params.filterByInertia = False
# Convexity parameters
params.filterByConvexity = False
# Other parameters
params.minRepeatability = 1

detector = cv.SimpleBlobDetector_create(params)

In [268]:
kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])
bimg = cv.filter2D(src=bimg, ddepth=-1, kernel=kernel)

In [276]:
blobs = detector.detect(bimg)
blob_img = cv.drawKeypoints(bimg, blobs, np.array([]), (0, 0, 255), cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv.imshow("blobs", blob_img)
cv.waitKey(0)
cv.destroyWindow("blobs")
cv.waitKey(1)

KeyboardInterrupt: 

In [38]:
kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])

images = os.listdir("positives/")

for im in images:
    if im.startswith(".") or im.startswith("resize"):
        continue
    limg = cv.imread(filename="positives/"+im)
    # limg = cv.filter2D(src=limg, ddepth=-1, kernel=kernel)
    blobs = detector.detect(limg)
    limg = cv.drawKeypoints(limg, blobs, np.array([]), (0, 0, 255), cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    cv.imshow("blobs", limg)
    k = cv.waitKey(0)
    cv.destroyWindow("blobs")
    cv.waitKey(1)

    if k == ord("q"):
        break